In [1]:

import os
import sys
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from itertools import product
from sklearn.preprocessing import RobustScaler, LabelEncoder

sys.path.extend([
                '../src',
                '../data',
                ])

from utils import run_nested_cv_3b

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("CPU")
    
seed_num = 42
random.seed(seed_num)
np.random.seed(seed_num)
tf.random.set_seed(seed_num)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

#data
left = pd.read_csv('../data/left.csv')
right = pd.read_csv('../data/right.csv')
affected = pd.read_csv('../data/affected.csv')
unaffected = pd.read_csv('../data/unaffected.csv')

#limb
right['limb'] = 1
left['limb'] = 2
affected['limb'] = 3
unaffected['limb'] = 4

affected['subject'] = affected['subject'] + 10
unaffected['subject'] = unaffected['subject'] + 10

le = LabelEncoder()
right['use_type_encoded'] = le.fit_transform(right['use_type'])
left['use_type_encoded'] = le.fit_transform(left['use_type'])
affected['use_type_encoded'] = le.fit_transform(affected['use_type'])
unaffected['use_type_encoded'] = le.fit_transform(unaffected['use_type'])

combined = pd.concat([unaffected, 
                      affected, 
                      right, 
                      left], ignore_index=True)

architecture = {
                'num_filters': [50, 25],
                'filter_sizes': [5, 5],
                'pool_sizes': [5, 5]
               }

CPU


In [2]:


approach = 3
filename = f'../results/approach3b/limb_nested_cv_generic.csv'

window_size = 100
epochs = 200
batch_size = 32
class_loss = False
plot_hist = False

error_func = 'mae'

X = combined[['ax', 'ay', 'az']]
y = combined['gnd']
subjects = combined['subject']
limbs = combined['limb']
usetypes = combined['use_type_encoded']

hp_grid =  {
        'activation': ['tanh', 'relu'],
        'kernel_init': [tf.keras.initializers.HeNormal(seed=42), 
                        tf.keras.initializers.GlorotNormal(seed=42), ],
        'bias_init': [tf.keras.initializers.Zeros()],
        'optimizer':['adam', 'nadam'],
        'loss': [tf.keras.losses.mean_squared_error,
                 tf.keras.losses.mean_absolute_error],
        'learning_rate': [1e-3],
        'batch_norm': [False, True]
        }

results_3b = run_nested_cv_3b(
    window_size,             
    X=X,                         
    y=y,                        
    subjects=subjects,           
    limbs=limbs,               
    usetypes=usetypes,          
    architecture=architecture,   
    hp_grid=hp_grid,
    epochs=epochs,                  
    batch_size=batch_size,              
    class_loss=False,           
    plot_hist=False,            
    error_func='mae'
)

print(np.mean(results_3b['youden_index']), np.std(results_3b['youden_index']))

2025-02-03 06:25:11.485253: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Mean validation score: 0.288
New best HP found: {'activation': 'tanh', 'batch_norm': False, 'bias_init': <keras.initializers.initializers_v2.Zeros object at 0x34c917d60>, 'kernel_init': <keras.initializers.initializers_v2.HeNormal object at 0x34c9172b0>, 'learning_rate': 0.001, 'loss': <function mean_squared_error at 0x31316f4c0>, 'optimizer': 'adam'} with score: 0.288
Mean validation score: 0.311
New best HP found: {'activation': 'tanh', 'batch_norm': False, 'bias_init': <keras.initializers.initializers_v2.Zeros object at 0x34c917d60>, 'kernel_init': <keras.initializers.initializers_v2.HeNormal object at 0x34c9172b0>, 'learning_rate': 0.001, 'loss': <function mean_squared_error at 0x31316f4c0>, 'optimizer': 'nadam'} with score: 0.311
Mean validation score: 0.297
Mean validation score: 0.272
Mean validation score: 0.285
Mean validation score: 0.290
Mean validation score: 0.243
Mean validation score: 0.222
Mean validation score: 0.160
Mean validation score: 0.243
Mean validation score: 

In [3]:
print("\nResults Summary:")
print(results_3b.groupby(['limb']).agg({
    'youden_index': ['mean', 'std'],
    'sensitivity': ['mean', 'std'],
    'specificity': ['mean', 'std']
}).round(3))


Results Summary:
     youden_index        sensitivity        specificity       
             mean    std        mean    std        mean    std
limb                                                          
1           0.326  0.161       0.928  0.051       0.399  0.156
2           0.270  0.062       0.837  0.114       0.433  0.156
3           0.232  0.278       0.779  0.141       0.453  0.245
4           0.377  0.253       0.820  0.155       0.558  0.164
